In [1]:
import importlib
import projet_madi as pm
from tqdm import tqdm

In [2]:
importlib.reload(pm)

<module 'projet_madi' from 'C:\\Users\\arian\\OneDrive\\Faculdade\\M2\\MADI\\Projet\\git\\projet_madi.py'>

In [3]:
# Définition des couleurs
red = "#F70B42"
green = "#1AD22C"
blue = "#0B79F7"
gray = "#E8E8EB"
orange = "#FFAA11"
darkgray = "#5E5E64"
white = "#FFFFFF"

nb_color = 4
color = [green, blue, red, darkgray]

## 1. Générateur d'instances et visualisation

In [6]:
# Création d'une grille
width = 10
height = 10
cost = [1, 2, 3, 4]
p = 1
proba_mur = 0.1
g = pm.Grille(height, width, tab_cost = cost, p = p, proba_mur = proba_mur)

In [11]:
# Visualisation de la grille (tourner cette cellule ouvre une nouvelle fenêtre)
v = pm.Visualisation(g, color)
v.view(mode = "couleur")

## 2. Recherche d'une trajectoire de moindre risque par itération de la valeur

### Exemple de calcul de stratégie sur une grille

On teste ici l'algorithme d'itération de la valeur implémenté dans la fonction `pol_valeur`. On peut varier les paramètres (`p`, `M`, taille de la grille et `gamma`) en modifiant la cellule ci-dessous.

In [12]:
width = 10
height = 10
p = 1
gamma = 0.9
M = 1000

In [13]:
# Création et visualisation de la grille avant de calculer la politique optimale
g = pm.Grille(height, width, tab_cost = cost, p = p, proba_mur = proba_mur)
v = pm.Visualisation(g, color)
v.view(mode = "couleur")

In [14]:
# Calcul de la stratégie optimale par itération de la valeur
strategy_valeur, nb_iter = pm.pol_valeur(g, gamma = gamma, M = M)
print("Nombre d'itérations : {}".format(nb_iter))

Nombre d'itérations : 19


In [16]:
# Affichage de la grille avec la stratégie
v.view(mode = "couleur", strategy = strategy_valeur)

In [21]:
# Nouveau calcul sur la même grille avec changement de la valeur de p
p = 0.6
g.p = p
strategy_valeur, nb_iter = pm.pol_valeur(g, gamma = gamma, M = M)
print("Nombre d'itérations : {}".format(nb_iter))
v.view(mode = "couleur", strategy = strategy_valeur)

Nombre d'itérations : 28


### Le cas d'une grille impossible

On considère maintenant ce qui se passe s'il n'y a pas de chemin possible entre la case initiale et la case but. Pour cela, on prend la grille précédente et on crée un mur dans la case cible.

In [31]:
M = 1000 # On revient à la valeur de départ de M
g.tab[-1, -1] = -1 # Mur sur la case cible
strategy_valeur, nb_iter = pm.pol_valeur(g, gamma = gamma, M = M)
print("Nombre d'itérations : {}".format(nb_iter))
v.view(mode = "couleur", strategy = strategy_valeur)

Nombre d'itérations : 111


### Effet du bonus $M$

On considère ici l'impact de $M$ sur la résolution d'une grille. On a fixé une grille particulière dans cette section pour illustrer quelques phénomènes qui se passent en fonction de $M$.

In [71]:
g = pm.Grille(7, 7, tab_cost = [1, 2, 3, 4], p = 0.6, proba_mur = 0)
g.tab[:, :] = 3
g.tab[2, :] = 2
g.tab[:, -3] = 2
g.tab[1, :] = 1
g.tab[:, -2] = 1
g.tab[0, :] = 0
g.tab[:, -1] = 0
v = pm.Visualisation(g, color)
v.view(mode = "couleur")

In [96]:
M = 2
gamma = 0.8
strategy_valeur, nb_iter = pm.pol_valeur(g, gamma = gamma, M = M)
print("Nombre d'itérations : {}".format(nb_iter))
v.view(mode = "couleur", strategy = strategy_valeur)

Nombre d'itérations : 19


### Temps d'exécution et nombre d'itérations de l'algorithme d'itération de la valeur

In [5]:
qtt = 50
tab_cost = [1, 2, 3, 4]
proba_mur = 0.1
repeat = 10
M = 1000

dict_temps_iter = {(p, gamma, height, width): None \
              for p in [0.6, 1]\
              for gamma in [0.5, 0.7, 0.9]\
              for height, width in [(10, 10), (10, 15), (15, 20)]}

dict_grilles = {}
for height, width in [(10, 10), (10, 15), (15, 20)]:
    dict_grilles[(height, width)] = []
    for _ in range(qtt):
        # Il faut que la création d'une grille avec une case but atteignable soit possible (proba_mur pas trop grand)
        while True:
            grille = pm.Grille(height, width, tab_cost, 1, None, proba_mur)
            if grille.est_possible():
                dict_grilles[(height, width)].append(grille)
                break

for k in tqdm(dict_temps_iter):
    p, gamma, height, width = k
    list_grille = dict_grilles[(height, width)]
    for g in list_grille:
        g.p = p
    temps = pm.tester_temps(pm.pol_valeur, list_grille, repeat, gamma = gamma, M = M, mode = "couleur")
    nb_iter = pm.tester_iterations(pm.pol_valeur, list_grille, gamma = gamma, M = M, mode = "couleur")
    dict_temps_iter[k] = (temps, nb_iter)

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [20:56<00:00, 69.78s/it]


In [6]:
for k in dict_temps_iter:
    temps, nb_iter = dict_temps_iter[k]
    print("{}: {:.1f} {:.1f}".format(k, temps*1000, nb_iter))

(0.6, 0.5, 10, 10): 37.0 18.8
(0.6, 0.5, 10, 15): 63.5 19.2
(0.6, 0.5, 15, 20): 157.1 19.5
(0.6, 0.7, 10, 10): 56.5 25.1
(0.6, 0.7, 10, 15): 111.0 32.5
(0.6, 0.7, 15, 20): 249.8 35.3
(0.6, 0.9, 10, 10): 78.0 33.2
(0.6, 0.9, 10, 15): 126.8 37.8
(0.6, 0.9, 15, 20): 342.9 50.0
(1, 0.5, 10, 10): 42.7 18.7
(1, 0.5, 10, 15): 63.6 19.0
(1, 0.5, 15, 20): 133.2 19.2
(1, 0.7, 10, 10): 41.4 20.1
(1, 0.7, 10, 15): 106.5 32.3
(1, 0.7, 15, 20): 238.8 35.1
(1, 0.9, 10, 10): 54.9 25.2
(1, 0.9, 10, 15): 98.0 29.9
(1, 0.9, 15, 20): 283.7 42.1


### Étude de l'impact du coût

On considère ici les variations du coût de chaque couleur, sous la forme de l'ajout d'une puissance $q$-ième à chaque coût et au bonus $M$ de la case but.

In [115]:
cost = [1, 2, 3, 4]
p = 0.8
width = 10
height = 10
proba_mur = 0.1

# Création et visualisation de la grille
g = pm.Grille(height, width, tab_cost = cost, p = p, proba_mur = proba_mur)
v = pm.Visualisation(g, color)
v.view(mode = "couleur")

In [147]:
M = 4
gamma = 0.9
q = 10
g.tab_cost = [c**q for c in cost]
strategy_valeur, nb_iter = pm.pol_valeur(g, gamma = gamma, M = M**q)
print("Nombre d'itérations : {}".format(nb_iter))
v.view(mode = "couleur", strategy = strategy_valeur)

Nombre d'itérations : 27


### Modélisation et résolution avec un autre critère de coût

Comme décrit dans le rapport, on considère ici que `tab_cost` vaut `[1, C, C**2, C**3]` avec une constante `C` égale à la quantité de cases de la grille.

In [156]:
p = 0.8
width = 15
height = 10
proba_mur = 0.1
C = width*height
cost = [1, C, C**2, C**3]

# Création et visualisation de la grille
g = pm.Grille(height, width, tab_cost = cost, p = p, proba_mur = proba_mur)
v = pm.Visualisation(g, color)
v.view(mode = "couleur")

In [158]:
M = C**2
gamma = 0.999
strategy_valeur, nb_iter = pm.pol_valeur(g, gamma = gamma, M = M)
print("Nombre d'itérations : {}".format(nb_iter))
v.view(mode = "couleur", strategy = strategy_valeur)

Nombre d'itérations : 144


## 3. Recherche d'une trajectoire de moindre risque par programmation linéaire

### Exemple de calcul de stratégie sur une grille

On teste ici les algorithmes de programmation linéaire implémentés dans les fonctions `pol_pl_mixte` et `pol_pl_pure`. On peut varier les paramètres (`p`, `M` et `gamma`) en modifiant la cellule ci-dessous. On fixe la taille de la grille à $10 \times 15$ dans cette section.

In [24]:
p = 1
gamma = 0.9
M = 100

In [25]:
# Définition d'une grille
width = 15
height = 10
cost = [1, 2, 3, 4]
proba_mur = 0.1

In [29]:
# Création de la grille 
g = pm.Grille(height, width, tab_cost = cost, p = p, proba_mur = proba_mur)

In [30]:
# Visualisation de la grille avant de calculer la politique optimale
v = pm.Visualisation(g, color)
v.view(mode = "couleur") 

In [31]:
# Calcul de la stratégie optimale par programmation linéaire - politique pure
strategy_pl_pure, val_pure = pm.pol_pl_pure(g, gamma = gamma, M = M)
# Calcul de la stratégie optimale par programmation linéaire - politique mixte
strategy_pl_mixte, val_mixte = pm.pol_pl_mixte(g, gamma = gamma, M = M)
# Calcul de la stratégie optimale par itération de la valeur
strategy_valeur, nb_iter = pm.pol_valeur(g, gamma = gamma, M = M)

print("Valeur de la fonction objectif (politiques pures) :", val_pure)
print("Valeur de la fonction objectif (politiques mixtes) :", val_mixte)
print("Nombre d'itérations de l'itération de la valeur :", nb_iter)

Valeur de la fonction objectif (politiques pures) : 314.67213928861224
Valeur de la fonction objectif (politiques mixtes) : 314.6721392886122
Nombre d'itérations de l'itération de la valeur : 24


In [32]:
# Affichage de la grille avec la stratégie par programmation linéaire en politique pure
v.view(mode = "couleur", strategy = strategy_pl_pure)

In [33]:
# Affichage de la grille avec la stratégie par itération de la valeur
v.view(mode = "couleur", strategy = strategy_valeur)

Il n'est pas possible de visualiser directement la stratégie mixte. Cependant, on peut l'afficher pour vérifier que, comme vu en cours, la stratégie mixte optimale correspond en effet à une stratégie pure : sur chaque case du tableau, la loi de probabilité sur les actions donne $100\%$ de probabilité pour une action et $0$ pour les autres. Les seules exceptions sont les murs : la politique optimale n'y est pas calculée et, comme valeur par défaut, on y met une loi uniforme sur les quatre actions.

In [34]:
print(strategy_pl_mixte)

[[[0.   0.   1.   0.  ]
  [0.   0.   1.   0.  ]
  [0.   0.   1.   0.  ]
  [0.   0.   1.   0.  ]
  [0.25 0.25 0.25 0.25]
  [0.   0.   1.   0.  ]
  [0.   0.   1.   0.  ]
  [0.   0.   0.   1.  ]
  [0.   0.   1.   0.  ]
  [0.25 0.25 0.25 0.25]
  [0.   1.   0.   0.  ]
  [0.   1.   0.   0.  ]
  [0.   0.   1.   0.  ]
  [0.   0.   1.   0.  ]
  [0.   0.   0.   1.  ]]

 [[0.   1.   0.   0.  ]
  [0.   1.   0.   0.  ]
  [0.   1.   0.   0.  ]
  [0.   1.   0.   0.  ]
  [0.   0.   1.   0.  ]
  [0.   1.   0.   0.  ]
  [0.   0.   1.   0.  ]
  [0.   0.   0.   1.  ]
  [0.   0.   0.   1.  ]
  [0.25 0.25 0.25 0.25]
  [0.   0.   1.   0.  ]
  [0.25 0.25 0.25 0.25]
  [0.   0.   1.   0.  ]
  [0.   0.   0.   1.  ]
  [0.   0.   0.   1.  ]]

 [[0.   0.   1.   0.  ]
  [0.   1.   0.   0.  ]
  [0.   0.   1.   0.  ]
  [0.   0.   1.   0.  ]
  [0.   0.   1.   0.  ]
  [0.   1.   0.   0.  ]
  [0.   0.   1.   0.  ]
  [0.25 0.25 0.25 0.25]
  [0.25 0.25 0.25 0.25]
  [0.   1.   0.   0.  ]
  [0.   0.   1.   0.  ]
  [0.   1.  

On peut retrouver la stratégie pure correspondante à cette stratégie mixte en cherchant l'action qui maximise la probabilité, ce qui se fait facilement en `numpy` à l'aide de la méthode `argmax`. Il est alors possible de visualiser la stratégie pure ainsi obtenue.

In [35]:
# Affichage de la grille avec la stratégie par programmation linéaire en politique mixte : direction avec la probabilité maximale
v.view(mode = "couleur", strategy = strategy_pl_mixte.argmax(2))

### Temps d'exécution et valeurs de la fonction objectif

In [4]:
qtt = 50
tab_cost = [1, 2, 3, 4]
proba_mur = 0.1
repeat = 10
M = 1000
height = 10
width = 15

In [5]:
dict_temps_val = {(p, gamma, pol): None \
              for p in [0.6, 1]\
              for gamma in [0.5, 0.7, 0.9]\
              for pol in ["pure", "mixte"]}

list_grille = []
for _ in range(qtt):
    # Il faut que la création d'une grille avec une case but atteignable soit possible (proba_mur pas trop grand)
    while True:
        grille = pm.Grille(height, width, tab_cost, 1, None, proba_mur)
        if grille.est_possible():
            list_grille.append(grille)
            break

In [6]:
for k in tqdm(dict_temps_val):
    p, gamma, pol = k
    fonction = getattr(pm, "pol_pl_" + pol)
    for g in list_grille:
        g.p = p
    temps = pm.tester_temps(fonction, list_grille, repeat, gamma = gamma, M = M, mode = "couleur")
    val_pol = pm.tester_iterations(fonction, list_grille, gamma = gamma, M = M, mode = "couleur")
    dict_temps_val[k] = (temps, val_pol)

  0%|                                                                                           | 0/12 [00:00<?, ?it/s]

Using license file C:\Users\arian\gurobi.lic
Academic license - for non-commercial use only - expires 2021-02-09


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [08:12<00:00, 41.07s/it]


In [7]:
for k in dict_temps_val:
    temps, val_pol = dict_temps_val[k]
    print("{}: {:.1f} {:.1f}".format(k, temps*1000, val_pol))

(0.6, 0.5, 'pure'): 151.6 46.0
(0.6, 0.5, 'mixte'): 42.9 46.0
(0.6, 0.7, 'pure'): 141.9 212.8
(0.6, 0.7, 'mixte'): 42.2 212.8
(0.6, 0.9, 'pure'): 136.0 3291.0
(0.6, 0.9, 'mixte'): 45.3 3291.0
(1, 0.5, 'pure'): 104.9 47.9
(1, 0.5, 'mixte'): 38.6 47.9
(1, 0.7, 'pure'): 105.3 218.3
(1, 0.7, 'mixte'): 39.6 218.3
(1, 0.9, 'pure'): 103.7 3320.4
(1, 0.9, 'mixte'): 42.2 3320.4


## 4. Recherche d'une trajectoire équilibrée

### Exemple de calcul de stratégie pour la minimisation de la consommation totale de ressources

### Exemple de calcul de stratégie pour la minimisation d'un critère équilibré

### Temps de résolution de la minimisation d'un critère équilibré

### Performances moyennes de la minimisation équilibrée sur une instance test

### Comparaison entre la minimisation d'un critère équilibré et de la consommation totale